In [1]:
import os
import requests
import pandas as pd
from multiprocessing.dummy import Pool

In [3]:
# get token
client_id = 'd4i7e6cxj7xmn7v1k2ck4olbxq8ml8'
client_secret = 'q01dqdn9ve74fwfh31ir5fvoh8a4fz'
token = None
if os.path.exists('token.txt'):
    with open('token.txt', 'r') as f:
        token = f.read()
    resp = requests.get('https://id.twitch.tv/oauth2/validate', headers={'Authorization': f'OAuth {token}'}).json()
    if resp['expires_in'] <= 10:
        token = None
if token is None:
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    resp = requests.post('https://id.twitch.tv/oauth2/token', params=params).json()
    token = resp['access_token']
    print('new token:', token)
    with open('token.txt', 'w') as f:
        f.write(token)
else:
    print('valid token:', token)
headers = {
    'Authorization': f'Bearer {token}',
    'Client-Id': client_id
}

valid token: z7kdzbred89ygtz94qb1hgb9iqw2qh


In [4]:
# get user_ids and followers
users = ['Fortnite', 'LCS', 'dota2ti', 'OverwatchLeague', 'ESL_CSGO']
params = '&'.join([f'login={user}' for user in users])
resp = requests.get(f'https://api.twitch.tv/helix/users?{params}', headers=headers).json()
user_ids = [item['id'] for item in resp['data']]
followers = []
for user, user_id in zip(users, user_ids):
    resp = requests.get('https://api.twitch.tv/helix/users/follows', params={'to_id': user_id}, headers=headers).json()
    followers.append(resp['total'])
    print(user, followers[-1], 'followers')



Fortnite 668726 followers
LCS 1784081 followers
dota2ti 4600246 followers
OverwatchLeague 4151896 followers
ESL_CSGO 1461190 followers


In [5]:
# scraping task
fields = ['title', 'published_at', 'view_count', 'duration']
def task(user: str, user_id: str, follower: int):
    df = pd.DataFrame(columns=fields)
    cursor = None
    while True:
        params = {'user_id': user_id, 'first': 100}
        if cursor is not None:
            params['after'] = cursor
        resp = requests.get('https://api.twitch.tv/helix/videos', params=params, headers=headers).json()
        data = resp['data']
        if len(data) == 0 or int(data[0]['published_at'][:4]) < 2018:
            break
        print(f'\r{user} {data[0]["published_at"]}\t\t', end='')
        cursor = resp['pagination']['cursor']
        for item in data:
            df.loc[item['id']] = [' '.join(str(item[field]).split()) for field in fields]
    df.to_csv(f'data/{user}_{follower}.csv')
def task_wrapper(arg):
    task(*arg)

# start parallel scraping
pool = Pool(len(users))
pool.map(task_wrapper, zip(users, user_ids, followers))
pool.close()
pool.join()


dota2ti 2018-01-22T06:32:39Z		